In [1]:
import os

#find the latest version of Pysparks
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

#Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

#Set Environment Variables
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-hadoop3'

#start a sparksession
import findspark
findspark.init()




Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,683 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [3]:
#Import Packages
from pyspark.sql import SparkSession
import time

#Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 Bucket into dataframe
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
#Create a temporary view of the DataFrame
df.createOrReplaceTempView('home_sales')


In [13]:
#5.1 What is the average price for a four-bedroom house sold for each year? Round off your answer to two decimal places.

sql = """
SELECT YEAR (date) AS YEAR, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YeAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

#


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [15]:
#5.2 What is the average price of a home for each year the home was built, that has three bedrooms and three bathrooms? Round off your answer to two decimal places.
sql = """
SELECT YEAR (date_built) AS YEAR, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(sql).show()

#

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [16]:
#o	What is the average price of a home for each year the home was built, that has three bedrooms,
#three bathrooms, two floors, and is greater than or equal to 2,000 square feet?
#Round off your answer to two decimal places.

sql = """
SELECT YEAR (date_built) AS YEAR, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors >= 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(sql).show()

#

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2010|    301130.93|
|2011|    290688.58|
|2012|    319456.36|
|2013|    310510.69|
|2014|     309821.9|
|2015|     307147.8|
|2016|    300851.65|
|2017|    293281.74|
+----+-------------+



In [27]:
#o	What is the average price of a home per "view" rating having an average home price greater than or equal to $350,000?
# Determine the run time for this query, and round off your answer to two decimal places.

start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()

print("---%s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

---0.5171046257019043 seconds ---


In [19]:
#6. Cache the temporary table
spark.sql('cache table home_sales')

DataFrame[]

In [21]:
#7 check if table is cached
spark.catalog.isCached('home_sales')

True

In [36]:
#8 Using the cached data, run the last query that calculates the average price of a home per "view" rating
#having an average home price greater than or equal to $350,000.
#Determine the runtime and compare it to uncached runtime.

start_time = time.time()

sql = """
SELECT view, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()

print("---%s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

---0.5205271244049072 seconds ---


In [41]:
#9	Partition by the "date_built" field on the formatted parquet home sales data.
df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')

In [45]:
# Read the parquet formatted data
p_df = spark.read.parquet('p_home_sales')

In [46]:
# 10 Create a temporary table for the parquet data
p_df.createOrReplaceTempView('p_home_sales')

In [54]:
# 11.	Run the last query that calculates the average price of a home per "view" rating having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to uncached runtime.
start_time = time.time()


sql = """
SELECT view, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM p_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
"""
spark.sql(sql).show()

print("---%s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
+----+-------------+
only showing top 20 rows

---0.5573275089263916 seconds ---


In [55]:
#12.	Uncache the home_sales temporary table

spark.sql('uncache table home_sales')

DataFrame[]

In [57]:
#12.	Verify that the home_sales temporary table is uncached using PySpark
spark.catalog.isCached('home_sales')

False